# Simple AI Agent

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
from traceloop.sdk import Traceloop
Traceloop.init(app_name="testing-openllmetry")

## Setting up the tools

This AI use web based document loader to load pages from langsmit documentation. 

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from IPython.display import display, Markdown

loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)
vector = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vector.as_retriever()

In [ ]:
retriever.invoke("What is Langsmith?")

In [5]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

tools = [retriever_tool]

## Setting up the models and agent

In [6]:
from langchain_google_vertexai import ChatVertexAI

model = ChatVertexAI(model="gemini-1.5-flash")

from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")

from langchain.agents import AgentExecutor
from langchain.agents import create_tool_calling_agent

agent = create_tool_calling_agent(model, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

## Add session storage

In [7]:

from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [8]:
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [9]:
import uuid

session_id = str(uuid.uuid4())

In [ ]:
response = agent_with_chat_history.invoke(
    {"input": "How langsmith can help with testing the LLM agent?"},
    config={"configurable": {"session_id": session_id}},
)

display(Markdown(response["output"]))
